# 얼굴인식 프로그램

### 딥러닝

CNN 모델  
train_model.py

In [2]:
import os
import cv2
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf

epochs = 15
image_w = 128
image_h = 128

data_path = "./data/archive"
hdf5_file = "./models/face.hdf5" # 모델 저장 위치

def model_train():
    images = []
    labels = []
    for label in os.listdir(data_path):
        label_path = os.path.join(data_path, label)
        if os.path.isdir(label_path):
            for filename in os.listdir(label_path):
                img_path = os.path.join(label_path, filename)
    
                pil_image = Image.open(img_path)
                np_image = np.array(pil_image)
                cv_image = cv2.cvtColor(np_image, cv2.COLOR_RGB2BGR)
                gray = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)
                
                image = cv2.resize(gray, (image_w, image_h))
    
                if image is not None:
                    images.append(image)
                    labels.append(label)
                    
    images = np.array(images)
    labels = np.array(labels)
    
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=42)
    # 라벨을 정수로 변환
    label_to_index = {label: idx for idx, label in enumerate(np.unique(labels))}
    y_train = np.array([label_to_index[label] for label in y_train])
    y_test = np.array([label_to_index[label] for label in y_test])
    
    # 데이터 정규화
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0
    
    # 새로운 모델 생성
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(image_w, image_h, 1)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
    
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(len(label_to_index), activation='softmax')
    ])
    
    # 모델 컴파일
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # 모델 요약 확인
    print(model.summary())
    
    # 모델 학습
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs)
    
    # 모델 저장하기
    model.save(hdf5_file)
    
    # 모델 평가
    score = model.evaluate(X_test, y_test)
    print('loss=', score[0])        # loss
    print('accuracy=', score[1])    # acc
    

if __name__=="__main__":
    model_train()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 128, 128, 64)      640       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 64, 64, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 64, 64, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 32, 32, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 32, 32, 256)       295168    
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 16, 16, 256)      

### 매니저 프로그램

정보 등록, 사진 찍기, 모델 트레이닝  
manager.py

In [5]:
import time
import os
import cv2
import tkinter as tk
from PIL import Image, ImageTk
import tkinter.scrolledtext as st
import threading

import train_model


class Manager():
    def __init__(self):
        filename = "./data/video/train4.mp4"
        self.cap = cv2.VideoCapture(filename)
        self.faces = []
        
        self.run()
    
    
    def join(self):
        self.num = self.num_entry.get()
        
        self.picture()
    
        self.window_join()
        
    
    def window_join(self):
        self.join_window = tk.Toplevel(self.window)
        self.join_window.title("Join")
        self.join_window.geometry("300x600")
        
        # 디렉토리에서 모든 파일들을 가져옴
        image_files = os.listdir(self.folder_path)

        # jpg 파일들의 이름에서 확장자를 제거하여 리스트에 추가
        image_names = [file.split('.jpg')[0] for file in image_files if file.endswith('.jpg')]

        self.label_image = tk.Label(self.join_window)
        self.label_image.grid(row=0, column=0, padx=3, pady=3)
        
        if image_names:
            image_button_frame = tk.Frame(self.join_window, width=280, height=280)
            image_button_frame.grid(row=1, column=0)
            
            for i, name in enumerate(image_names):
                image_path = self.folder_path +"/"+ name +".jpg"
                image = Image.open(image_path)
                image = image.resize((64,64), Image.LANCZOS)
                photo = ImageTk.PhotoImage(image)
                
                image_button = tk.Button(image_button_frame, image=photo, command=lambda image_path=image_path: self.on_image_button_click(image_path))
                image_button.image = photo
                image_button.grid(row= i//3 +1, column= i%3)
                
            join_ok_button = tk.Button(self.join_window, text="ok", command=self.join_ok)
            join_ok_button.grid(row=2, column=0)
            
        else:
            label_no_image = tk.Label(self.join_window, text="no image")
            label_no_image.grid(row=1, column=0)
        
        cancel_button = tk.Button(self.join_window, text="cancel", command=lambda: self.join_window.destroy())
        cancel_button.grid(row=3, column=0)
        
    def on_image_button_click(self, image_path):
            self.display_selected_image(image_path)
    
    def display_selected_image(self, image_path):
        image = Image.open(image_path)
        image = image.resize((256, 256), Image.LANCZOS)
        photo = ImageTk.PhotoImage(image)
        self.label_image.config(image=photo)
        self.label_image.image = photo  # 유지하기 위해 참조를 저장해야 함
    
    def join_thread(self):
        if self.num_entry.get():
            thread = threading.Thread(target=self.join)
            thread.start()
    
    def picture(self):
        wait_time = 2
        image_w = 128
        image_h = 128
        
        # 이미지를 저장할 폴더 경로 설정
        self.folder_path = "./data/archive/" + self.num
        
        if not os.path.exists(self.folder_path):
            os.makedirs(self.folder_path)  # 폴더가 없는 경우 폴더 생성
        
        for i in range(10):
            image_path = os.path.join(self.folder_path, f"{self.num}_{i+1}.jpg")  # 이미지 파일 경로 설정   
            
            for t in range(wait_time):
                self.MyLog(f"{wait_time - t}s...")
                time.sleep(1)
            
            self.MyLog(f"{i+1}번 촬영")
            
            if len(self.faces) > 0:
                (x,y,w,h) = self.faces[0]
                x -= int(0.2 * w)
                y -= int(0.2 * h)
                w = int(1.4 * w)
                h = int(1.4 * h)
                
                roi = self.blur[y:y+h, x:x+w]
                # roi 이미지의 크기가 유효한지 확인
                if roi.shape[0] > 0 and roi.shape[1] > 0:
                    
                    face_roi = cv2.resize(roi, (image_w, image_h))
        
                    cv2.imwrite(image_path, face_roi)
                    
                    self.MyLog(f"picture success...{i+1}")
                
                else:
                    self.MyLog(f"failed picture...{i+1}")
            
        
    def join_ok(self):
        # 모델 학습
        self.MyLog("model training...")
        train_model.model_train()
    
    
    def video_update(self):
        skip_frames = 20  # 얼굴 감지 수행 프레임

        ret, frame = self.cap.read()
        
        if ret:
            photo = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # 매 skip_frames번째 프레임에서만 얼굴 감지 수행
            if self.cap.get(cv2.CAP_PROP_POS_FRAMES) % skip_frames == 0:
                # OpenCV의 얼굴 검출을 위한 분류기를 로드합니다.
                face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
                gray = cv2.cvtColor(photo, cv2.COLOR_RGB2GRAY)
                
                self.blur = cv2.GaussianBlur(gray, (5,5), 0)
    
                # 얼굴 감지
                self.faces = face_cascade.detectMultiScale(self.blur, scaleFactor=1.1, minNeighbors=3, minSize=(60, 60))
                
            # 감지된 얼굴 주위에 사각형 그리기
            for (x, y, w, h) in self.faces:
                # 머리 크기까지 사각형 확장
                x -= int(0.2 * w)
                y -= int(0.2 * h)
                w = int(1.4 * w)
                h = int(1.4 * h)
                 
                cv2.rectangle(photo, (x, y), (x+w, y+h), (0, 255, 0), 2)

            img = Image.fromarray(photo)
            
            imgTK = ImageTk.PhotoImage(image=img) # tkinter 호환 이미지로 변경
            
            self.canvas.config(width=imgTK.width(), height=imgTK.height())

            self.canvas.create_image(0,0,anchor="nw",image=imgTK)
            self.canvas.image = imgTK
        else:
            self.cap.set(cv2.CAP_PROP_POS_FRAMES, 0) # 동영상이 끝났으면 다시 시작

        self.window.after(20, self.video_update)
    
    
    def run(self):
        self.window = tk.Tk()
        self.window.title("manager")
        
        self.canvas = tk.Canvas(self.window)
        self.canvas.grid(row=0, column=0, rowspan=2, padx=3, pady=3)
        
        
        self.join_frame = tk.Frame(self.window, width=100)
        self.join_frame.grid(row=0, column=1, padx=3, pady=3)
        
        self.num_label = tk.Label(self.join_frame, text="num :")
        self.num_label.pack()
        
        self.num_entry = tk.Entry(self.join_frame)
        self.num_entry.pack()
        
        self.join_button = tk.Button(self.join_frame, text="Join", command=self.join_thread)
        self.join_button.pack()
        
        self.textLog = st.ScrolledText(self.window,
                                    width = 30,
                                    height = 8,
                                    font = ("Times New Roman",10))
        self.textLog.grid(row=1, column=1, padx=3, pady=3)
        
        self.video_update()
        
        self.window.mainloop()

    def MyLog(self, msg):
        # scrolled text print...
        self.textLog.insert(tk.INSERT, msg + "\r\n")
        self.textLog.see("end")



if __name__ == "__main__":
    Manager()

![ss_manager.png](ss_manager.png)

![ss_manager_picture.png](ss_manager_picture.png)

### 얼굴인식 프로그램

얼굴 인식, 모델 예측  
main.py

In [8]:
import os
import tkinter as tk
import tkinter.scrolledtext as st
import tkinter.filedialog as fd
import cv2
import numpy as np
from PIL import Image
from PIL import ImageTk
from tensorflow.keras.models import load_model


filename = "./data/video/test.mp4"
cap = cv2.VideoCapture(filename)
# cv2.VideoCapture(0) 0은 내장된 기본캠, 외부 연결 카메라는 1부터 시작

hdf5_file = "./models/face.hdf5"
model = load_model(hdf5_file)
image_w = 128
image_h = 128

skip_frames = 20  # 얼굴 감지 수행 프레임
faces = []
result_face_recognize = ""


def MyLog(msg):
    # scrolled text print...
    textLog.insert(tk.INSERT, msg + "\r\n")
    textLog.see("end")

def update_video():
    global faces, result_face_recognize
    
    if not paused:
        ret, frame = cap.read()
        if ret:
            photo = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            
            # 매 skip_frames번째 프레임에서만 얼굴 감지 수행
            if cap.get(cv2.CAP_PROP_POS_FRAMES) % skip_frames == 0:
                # OpenCV의 얼굴 검출을 위한 분류기를 로드합니다.
                face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
                gray = cv2.cvtColor(photo, cv2.COLOR_RGB2GRAY)
                
                blur = cv2.GaussianBlur(gray, (5,5), 0)
                # 얼굴 감지
                faces = face_cascade.detectMultiScale(blur, scaleFactor=1.1, minNeighbors=3, minSize=(30, 30))
                
                if len(faces) == 1:
                    (x,y,w,h) = faces[0]
                    x -= int(0.2 * w)
                    y -= int(0.2 * h)
                    w = int(1.4 * w)
                    h = int(1.4 * h)
                    
                    roi = blur[y:y+h, x:x+w]
                    # roi 이미지의 크기가 유효한지 확인
                    if roi.shape[0] > 0 and roi.shape[1] > 0:
                        # 이미지를 모델 입력에 맞게 변환
                        pre = cv2.resize(roi, (image_w, image_h))
                        
                        face_roi = np.expand_dims(pre, axis=0) # 배치 차원 추가

                        predictions = model.predict(face_roi)[0]
                        #print(predictions) #예측 결과 프린트
                        
                        # 가장 높은 확률을 가진 클래스의 인덱스 추출
                        predicted_class_index = np.argmax(predictions)
                        
                        labels =  os.listdir("./data/archive")
                        label = labels[predicted_class_index]
                        
                        # 학습된 얼굴 클래스에 대한 확률 추출
                        known_face_probability = predictions[predicted_class_index]
                        
                        # 이상 감지 임계값을 설정하여 이상 감지 수행
                        threshold = 0.5  # 적절한 임계값으로 조정
                        if known_face_probability < threshold:
                            MyLog("이상 감지: 학습되지 않은 얼굴입니다.")
                            result_face_recognize = "Not training"
                        if label == "00":
                            MyLog("Unknown face")
                            result_face_recognize = "Unknown face"
                        else:
                            MyLog("학습된 얼굴입니다. " + label)
                            result_face_recognize = "No. "+label

            # 감지된 얼굴 주위에 사각형 그리기
            for (x, y, w, h) in faces:
                # 머리 크기까지 사각형 확장
                x -= int(0.2 * w)
                y -= int(0.2 * h)
                w = int(1.4 * w)
                h = int(1.4 * h)
                
                cv2.rectangle(photo, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(photo, result_face_recognize, (x+5,y+h-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            img = Image.fromarray(photo) # return photo object
            
            imgTK = ImageTk.PhotoImage(image=img) # tkinter 호환 이미지로 변경
            
            canvas.config(width=imgTK.width(), height=imgTK.height())

            canvas.create_image(0,0,anchor="nw",image=imgTK)
            canvas.image = imgTK
        
        else:
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0) # 동영상이 끝났으면 다시 시작

        window.after(20, update_video)

paused = False
play_button_text = "Pause"

def toggle_play_pause():
    global paused, play_button_text
    paused = not paused
    
    if play_button_text == "Pause":
        play_button_text = "Play"
    else:
        play_button_text = "Pause"
    
    play_button.config(text=play_button_text)
    
def on_canvas_click(event):
    toggle_play_pause()

def play_pause():
    toggle_play_pause()

def backward():
    cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) - 10 * cap.get(cv2.CAP_PROP_FPS))
    check_pause()

def forward():
    if (cap.get(cv2.CAP_PROP_POS_FRAMES) + 10 * cap.get(cv2.CAP_PROP_FPS)) < (cap.get(cv2.CAP_PROP_FRAME_COUNT) - 1):
        cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_POS_FRAMES) + 10 * cap.get(cv2.CAP_PROP_FPS))
        check_pause()
    else:
        latest()

def latest():
    cap.set(cv2.CAP_PROP_POS_FRAMES, cap.get(cv2.CAP_PROP_FRAME_COUNT) - 1)
    check_pause()

def close():
    if cap.isOpened():
        cap.release()
    window.destroy()

def on_key_press(event):
    if event.keysym == "space":
        toggle_play_pause()
    
    if event.keysym == "Left":
        backward()

    if event.keysym == "Right":
        forward()
        
def check_pause():
    if paused == True:
        toggle_play_pause()
    
def video_open():
    global filename, cap
    global faces, result_face_recognize
    
    filename = fd.askopenfilename(initialdir='./data/video/',title="select a file",
                                            filetypes=(("Video files", "*.mp4;*.avi"),
                                                       ("all files","*.*")))
    if filename:
        if cap is not None and cap.isOpened():
            cap.release()
            faces = []
            result_face_recognize = ""
            
        cap = cv2.VideoCapture(filename)
        check_pause()
        update_video()



# Tk 객체 생성
window = tk.Tk()
window.title("face")

canvas = tk.Canvas(window, bg="black")
canvas.grid(row=0, column=0, padx=3, pady=3)
# 캔버스에 클릭 이벤트 바인딩
canvas.bind("<Button-1>", on_canvas_click)


play_frame = tk.Frame(window)
play_frame.grid(row=1, column=0, padx=3, pady=3)

back_button = tk.Button(play_frame, text="Back", width=10)
back_button.grid(row=0, column=0, padx=3, pady=3)

backward_button = tk.Button(play_frame, text="Backward", width=10, command=backward)
backward_button.grid(row=0, column=1, padx=3, pady=3)

play_button = tk.Button(play_frame, text=play_button_text, width=10, command=play_pause)
play_button.grid(row=0, column=2, padx=3, pady=3)

forward_button = tk.Button(play_frame, text="Forward", width=10, command=forward)
forward_button.grid(row=0, column=3, padx=3, pady=3)

latest_button = tk.Button(play_frame, text="Latest", width=10, command=latest)
latest_button.grid(row=0, column=4, padx=3, pady=3)

# 키보드 바인딩
window.bind("<space>", on_key_press)
window.bind("<Left>", on_key_press)
window.bind("<Right>", on_key_press)

textLog = st.ScrolledText(window,
                            width = 30,
                            height = 8,
                            font = ("Times New Roman",10))
textLog.grid(row=0, column=1, padx=3, pady=3)

video_open_button = tk.Button(window, text="video_open", command=video_open)
video_open_button.grid(row=1, column=1, padx=3, pady=3)


update_video()

window.protocol("WM_DELETE_WINDOW", close)

window.mainloop()

1/1 [==============================] - 0s 16ms/step


![ss_face_recognize_test.png](ss_face_recognize_test.png)

![ss_face_recognize_train.png](ss_face_recognize_train.png)